In [9]:
import pandas as pd
import numpy as np

# Lire les fichiers de données
train_df = pd.read_csv('data/train.csv')
client_df = pd.read_csv('data/client.csv')
electricity_prices_df = pd.read_csv('data/electricity_prices.csv')
gas_prices_df = pd.read_csv('data/gas_prices.csv')

In [10]:
train_df_sample = train_df.sample(80000)


In [11]:
train_df_sample

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id
565100,15,1,1,0.000,0,2022-03-05 03:00:00,185,565100,59
1192574,8,0,3,0.000,0,2022-09-17 01:00:00,381,1192574,32
154600,13,1,1,0.000,0,2021-10-23 01:00:00,52,154600,63
426586,10,1,3,2.390,0,2022-01-20 09:00:00,141,426586,42
1845884,3,0,1,187.275,0,2023-04-07 16:00:00,583,1845884,11
...,...,...,...,...,...,...,...,...,...
1106903,5,0,1,14.661,1,2022-08-21 06:00:00,354,1106903,19
770975,0,1,1,630.948,1,2022-05-08 01:00:00,249,770975,4
468166,14,1,1,0.000,0,2022-02-02 16:00:00,154,468166,55
1751267,7,0,1,221.939,1,2023-03-08 18:00:00,553,1751267,25


In [12]:
electricity_prices_df["date"] = pd.to_datetime(electricity_prices_df["forecast_date"])
electricity_prices_df["date"] = electricity_prices_df["date"].apply(lambda x : x.strftime("%d-%m-%y"))
electricity_prices_df["date"]

0        01-09-21
1        01-09-21
2        01-09-21
3        01-09-21
4        01-09-21
           ...   
15281    30-05-23
15282    30-05-23
15283    30-05-23
15284    30-05-23
15285    30-05-23
Name: date, Length: 15286, dtype: object

In [13]:
client_df["date_str"] = client_df["date"].apply(lambda x : str(x))
electricity_prices_df["date_str"] = electricity_prices_df["date"].apply(lambda x : str(x))

In [14]:
electricity_prices_df

,forecast_date,euros_per_mwh,origin_date,data_block_id,date,date_str
0,2021-09-01 00:00:00,92.51,2021-08-31 00:00:00,1,01-09-21,01-09-21
1,2021-09-01 01:00:00,88.90,2021-08-31 01:00:00,1,01-09-21,01-09-21
2,2021-09-01 02:00:00,87.35,2021-08-31 02:00:00,1,01-09-21,01-09-21
3,2021-09-01 03:00:00,86.88,2021-08-31 03:00:00,1,01-09-21,01-09-21
4,2021-09-01 04:00:00,88.43,2021-08-31 04:00:00,1,01-09-21,01-09-21
...,...,...,...,...,...,...
15281,2023-05-30 19:00:00,82.10,2023-05-29 19:00:00,637,30-05-23,30-05-23
15282,2023-05-30 20:00:00,150.85,2023-05-29 20:00:00,637,30-05-23,30-05-23
15283,2023-05-30 21:00:00,82.10,2023-05-29 21:00:00,637,30-05-23,30-05-23
15284,2023-05-30 22:00:00,82.09,2023-05-29 22:00:00,637,30-05-23,30-05-23


In [15]:
train_df["price_elec"] = -1

for unique_date in train_df["datetime"].unique()[:10]:
  # Récuperer le prix de l'electricite a ce jour la
  df_elec_filtered = electricity_prices_df[electricity_prices_df["origin_date"] == unique_date]
  if len(df_elec_filtered) != 0:
    if len(df_elec_filtered) == 1:
      elec_price = df_elec_filtered["euros_per_mwh"]
    else : 
      elec_price = df_elec_filtered.iloc[0]["euros_per_mwh"]

    # Pour toutes les lignes de df_train a ce jour la, insérer le prix
    df_train_filtered = train_df[train_df["datetime"] == unique_date]
    df_train_filtered.loc[:,"price_elec"] = elec_price

In [16]:
train_df.describe()

,county,is_business,product_type,target,is_consumption,data_block_id,row_id,prediction_unit_id,price_elec
count,2.018352e+06,2.018352e+06,2.018352e+06,2.017824e+06,2018352.0,2.018352e+06,2.018352e+06,2.018352e+06,2018352.0
mean,7.297034e+00,5.368261e-01,1.898927e+00,2.748556e+02,0.5,3.218746e+02,1.009176e+06,3.304538e+01,-1.0
std,4.780990e+00,4.986421e-01,1.081766e+00,9.095024e+02,0.5,1.826343e+02,5.826482e+05,1.959059e+01,0.0
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,-1.0
25%,3.000000e+00,0.000000e+00,1.000000e+00,3.780000e-01,0.0,1.660000e+02,5.045878e+05,1.600000e+01,-1.0
50%,7.000000e+00,1.000000e+00,2.000000e+00,3.113300e+01,0.5,3.230000e+02,1.009176e+06,3.300000e+01,-1.0
75%,1.100000e+01,1.000000e+00,3.000000e+00,1.802062e+02,1.0,4.790000e+02,1.513763e+06,5.000000e+01,-1.0
max,1.500000e+01,1.000000e+00,3.000000e+00,1.548027e+04,1.0,6.370000e+02,2.018351e+06,6.800000e+01,-1.0


In [17]:
def find_time(group):
  electricity_prices_df[electricity_prices_df["origin_date"] == group.iloc[0]["datetime"]].iloc[0]["euros_per_mwh"]
  train_df.groupby("datetime")["electricity price"] = train_df.groupby("datetime").apply(find_time)

In [22]:
import time

def find_electricity_price(line):
  if int(line.name) % 2000 == 0 : 
    print(line.name, "/", train_df.shape[0],
          (line.name/train_df.shape[0])*100, "%", end="\r")
    time.sleep(2)
  return electricity_prices_df[electricity_prices_df["origin_date"] == line["datetime"]].iloc[0]["euros_per_mwh"]

train_df["euros_per_mwh"] = train_df.apply(find_electricity_price, axis = 1)
train_df

IndexError: single positional indexer is out-of-bounds

In [51]:
df_elec_filtered

,forecast_date,euros_per_mwh,origin_date,data_block_id,date_str,date
33,2021-09-02 09:00:00,130.8,2021-09-01 09:00:00,2,02-09-21,02-09-21


In [10]:
electricity_prices_df

,forecast_date,euros_per_mwh,origin_date,data_block_id,date_str,date
0,2021-09-01 00:00:00,92.51,2021-08-31 00:00:00,1,2021-09-01 00:00:00,01-09-21
1,2021-09-01 01:00:00,88.90,2021-08-31 01:00:00,1,2021-09-01 01:00:00,01-09-21
2,2021-09-01 02:00:00,87.35,2021-08-31 02:00:00,1,2021-09-01 02:00:00,01-09-21
3,2021-09-01 03:00:00,86.88,2021-08-31 03:00:00,1,2021-09-01 03:00:00,01-09-21
4,2021-09-01 04:00:00,88.43,2021-08-31 04:00:00,1,2021-09-01 04:00:00,01-09-21
...,...,...,...,...,...,...
15281,2023-05-30 19:00:00,82.10,2023-05-29 19:00:00,637,2023-05-30 19:00:00,30-05-23
15282,2023-05-30 20:00:00,150.85,2023-05-29 20:00:00,637,2023-05-30 20:00:00,30-05-23
15283,2023-05-30 21:00:00,82.10,2023-05-29 21:00:00,637,2023-05-30 21:00:00,30-05-23
15284,2023-05-30 22:00:00,82.09,2023-05-29 22:00:00,637,2023-05-30 22:00:00,30-05-23


In [67]:
train_df_sample["price_elec"] = -1

for unique_date in train_df_sample["datetime"].unique()[:10]:
  # Récuperer le prix de l'electricite a ce jour la
  df_elec_filtered = electricity_prices_df[electricity_prices_df["origin_date"] == unique_date]
  if len(df_elec_filtered) != 0:
    if len(df_elec_filtered) == 1:
      elec_price = df_elec_filtered["euros_per_mwh"]
    else : 
      elec_price = df_elec_filtered.iloc[0]["euros_per_mwh"]

    # Pour toutes les lignes de df_train a ce jour la, insérer le prix
    df_train_filtered = train_df_sample[train_df_sample["datetime"] == unique_date]
    df_train_filtered.loc[:,"price_elec"] = elec_price

In [68]:
df_elec_filtered

,forecast_date,euros_per_mwh,origin_date,data_block_id,date_str,date
8082,2022-08-03 19:00:00,541.54,2022-08-02 19:00:00,337,03-08-22,03-08-22


In [69]:
train_df_sample.describe()

,county,is_business,product_type,target,is_consumption,data_block_id,row_id,prediction_unit_id,price_elec
count,80000.000000,80000.000000,80000.000000,79973.000000,80000.000000,80000.000000,8.000000e+04,80000.000000,80000.0
mean,7.293637,0.539737,1.892000,274.237140,0.503800,322.887437,1.012389e+06,33.010663,-1.0
std,4.779867,0.498422,1.084274,908.976509,0.499989,182.509022,5.822921e+05,19.595952,0.0
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.100000e+01,0.000000,-1.0
25%,3.000000,0.000000,1.000000,0.405000,0.000000,167.000000,5.092528e+05,16.000000,-1.0
50%,7.000000,1.000000,2.000000,31.851000,1.000000,324.000000,1.012174e+06,33.000000,-1.0
75%,11.000000,1.000000,3.000000,182.373000,1.000000,480.000000,1.517621e+06,50.000000,-1.0
max,15.000000,1.000000,3.000000,14677.620000,1.000000,637.000000,2.018341e+06,68.000000,-1.0


In [71]:
def find_time(group):
  electricity_prices_df[electricity_prices_df["origin_date"] == group.iloc[0]["datetime"]].iloc[0]["euros_per_mwh"]
  train_df_sample.groupby("datetime")["electricity price"] = train_df_sample.groupby("datetime").apply(find_time)

In [72]:
print(train_df_sample.head())

         county  is_business  product_type   target  is_consumption  \
361532        2            0             1    0.000               0   
1200843       3            1             1   38.385               1   
555380        4            1             3    0.000               0   
930556       15            1             1  324.223               0   
1947593       2            1             1   53.700               1   

                    datetime  data_block_id   row_id  prediction_unit_id  \
361532   2021-12-30 12:00:00            120   361532                  62   
1200843  2022-09-19 15:00:00            383  1200843                  13   
555380   2022-03-02 02:00:00            182   555380                  18   
930556   2022-06-26 15:00:00            298   930556                  59   
1947593  2023-05-09 22:00:00            615  1947593                  65   

         price_elec  
361532           -1  
1200843          -1  
555380           -1  
930556           -1  
194759

In [3]:
import time

def find_electricity_price(line):
    if int(line.name) % 2000 == 0: 
        print("Processing row", line.name, "/", train_df_sample.shape[0])
        time.sleep(2)

    origin_date = line["datetime"]
    filtered_prices = electricity_prices_df[electricity_prices_df["origin_date"] == origin_date]

    if not filtered_prices.empty:
        euros_per_mwh = filtered_prices.iloc[0]["euros_per_mwh"]
        return euros_per_mwh
    else:
        return None

train_df_sample = train_df_sample.reset_index(drop=True)

train_df_sample["euros_per_mwh"] = train_df_sample.apply(find_electricity_price, axis=1)

print(train_df_sample)


NameError: name 'train_df_sample' is not defined

In [2]:
train_df_sample["euros_per_mwh"]

NameError: name 'train_df_sample' is not defined

In [1]:
train_df_sample.to_csv("./train_df_sample.csv")

NameError: name 'train_df_sample' is not defined